In [0]:


import os
import sys
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir,".."))
src0_root = os.path.abspath(os.path.join(project_root,".."))
#src_root = os.path.abspath(os.path.join(src0_root,"..")) + "/src"
src_root = os.path.abspath(os.path.join(src0_root,"..")) 

sys.path.append(src_root)

#print(sys.path)
print(project_root)
print(src0_root)
print(src_root)

#/Workspace/Shared/.bundle/dab_project/production/files/citibike_etl/notebooks
#/Workspace/Shared/.bundle/dab_project/production/files/citibike_etl/src

In [0]:
#from citibike.citibike_utils import get_trip_duration_mins
#from utils.datetime_utils import timestamp_to_date_col
"""
from src.citibike.citibike_utils import get_trip_duration_mins
from src.utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit
"""

from src.citibike.citibike_utils import get_trip_duration_mins
from src.utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [0]:


pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

"""
pipeline_id = 111
run_id = 222
task_id = 444
processed_timestamp = "2025-11-29"
catalog = "citibike_dev"
"""


In [0]:

table_to_read = f"{catalog}.01_bronze.jc_citibike"
df = spark.read.table(table_to_read)


In [0]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [0]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [0]:
df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

In [0]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )

In [0]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.02_silver.jc_citibike")

In [0]:
display(df)